# Importing Data and Libraries

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
con = sqlite3.connect('data/im.db')
cur = con.cursor()

In [ ]:
bmg = pd.read_csv('data/bom.movie_gross.csv')
mb = pd.read_csv('data/tn.movie_budgets.csv')
m = pd.read_csv('data/tmdb.movies.csv')

In [ ]:
rv = pd.read_csv('data/rt.reviews.tsv', delimiter = '\t', encoding = 'latin') 
# needed to change the encoding as it was throwing an error message about it
# tsv file, so needed to add a delimiter of tab

In [ ]:
mi = pd.read_csv('data/rt.movie_info.tsv', delimiter = '\t')

In [ ]:
imdb_rated = pd.read_csv('data/IMDB_rated_only.csv')

# Exploring the im.db Data

In [ ]:
%%bash

sqlite3 data/im.db
.tables

#checking the table names

In [ ]:
%%bash

sqlite3 data/im.db
.schema

#checking schema

In [ ]:
query_result = pd.read_sql('''
    SELECT *
    FROM 
        movie_ratings AS mr 
        INNER JOIN movie_basics AS mb
            ON mb.movie_id = mr.movie_id
    
    ORDER BY "averagerating" DESC

    LIMIT 50
    
''', con)

query_result

In [ ]:
query_result = pd.read_sql('''
    SELECT DISTINCT *
    FROM directors, movie_akas, movie_ratings, principals,  
          known_for, movie_basics, persons, writers 
    LIMIT 5
''', con)

query_result

In [ ]:
con.close()

# Cleaning CSV files

In [ ]:
bmg = bmg.reset_index()

In [ ]:
bmg['foreign_gross'] = bmg['foreign_gross'].str.replace(',','')

In [ ]:
bmg['foreign_gross'] = bmg['foreign_gross'].astype(float)

In [ ]:
mb['domestic_gross'] = mb['domestic_gross'].str.replace('$','')
mb['domestic_gross'] = mb['domestic_gross'].str.replace(',','')
mb['domestic_gross'] = mb['domestic_gross'].astype(int)

In [ ]:
mb['worldwide_gross'] = mb['worldwide_gross'].str.replace('$','')

In [ ]:
mb['worldwide_gross'] = mb['worldwide_gross'].str.replace(',','')

In [ ]:
mb['worldwide_gross'] = mb['worldwide_gross'].astype(int)

In [ ]:
mb['foreign_gross'] = mb['worldwide_gross'] - mb['domestic_gross']

In [ ]:
mb['release_date'] = pd.to_datetime(mb['release_date'])

In [ ]:
mb['release_year'] = mb['release_date'].dt.year

In [ ]:
m['release_date'] = pd.to_datetime(m['release_date'])

In [ ]:
mb['release_year'] = mb['release_date'].dt.year

In [ ]:
m['release_year'] = m['release_date'].dt.year

In [ ]:
print(bmg['year'].dtype)

# Exploring the Movie Gross file

In [ ]:
bmg.info()
# looks like foreign_gross is a string when it should be an integer

In [ ]:
bmg.isna().sum()
#foreign gross has a ton of null values

In [ ]:
mb.notna().sum()

# Exploring Movie Budgets file

In [ ]:
mb.head()

# Combining Data: bmg, mb, m

In [ ]:
# tried this out mbm = m.merge(mb, how = 'left', left_on =['original_title', 'release_date'], right_on =['movie', 'release_date'])

In [ ]:
#tried this out csvmerge = mbm.merge(bmg, how = 'left', left_on =['original_title', 'release_year'], right_on =['title', 'year'])

In [ ]:
csvmerge =  m.merge(mb, how = 'left', 
                    left_on =['original_title', 'release_year'], 
                    right_on =['movie', 'release_year']).merge(bmg, how = 'left',
                                left_on =['original_title', 'release_year'], 
                                right_on =['title', 'year'])

In [ ]:
#print(m.count())
#print(mb.count())
#print(bmg.count())
csvmerge.count()
#checking to see if merge worked ok and how many rows merged well together

In [ ]:
csvmerge.tail(1)

In [ ]:
print(bmg.columns)
print(csvmerge.columns)
csvmerge.info()

In [ ]:
allcsvs_merged = imdb_rated.merge(csvmerge, how = 'left',
                 left_on = ['primary_title', 'start_year'], 
                 right_on = ['movie', 'release_year']).dropna().drop_duplicates(subset = ['movie','start_year'])

# Exploring Combined CSV tables

In [ ]:
allcsvs_merged.info()

In [ ]:
allcsvs_merged.tail(1)

In [ ]:
allcsvs_merged.shape

In [ ]:
allcsvs_merged.columns

In [ ]:
allcsvs_merged.sort_values(by='worldwide_gross')

In [ ]:
allcsvs_merged.sample(4)

In [ ]:
pd.set_option('display.max_rows', None)
allcsvs_merged.sort_values(by='primary_title')

In [ ]:
pd.set_option('display.max_rows', 50)

In [ ]:
allcsvs_merged.columns

# Cleaning the Merged Data

In [ ]:
cleancm = allcsvs_merged[['title_x', 'genres', 'original_language', 'foreign_gross_x', 'foreign_gross_y', 'worldwide_gross', 
                     'release_date_x', 'domestic_gross_x', 'domestic_gross_y', 
                     'production_budget', 'popularity', 'vote_average',
                     'vote_count', 'release_year', 'id_y', 'release_date_y',
                       'studio',
                      'averagerating', 'numvotes',
                       'runtime_minutes', ]]
cleancm.head()

In [ ]:
cleancm.info()

In [ ]:
cleancm.sort_values(by='foreign_gross_x', ascending=True).head(20)
#cleancm.sort_values(by='foreign_gross_y', ascending=True).head(20)

# compared foreign_gross_x values with foreign_gross_y values
# looks like there is some bad data in both
# Seems like there is less bad data in is foreign_gross_x
# need to drop two rows of bad data in it, and use foreign_gross_x for visualizations

In [ ]:
cleancm = cleancm.drop([67902, 37757])

In [ ]:
cleancm.sort_values(by='foreign_gross_x', ascending=False).head(20)

## Making Merged DataFrame into CSV

In [ ]:
cleancm.to_csv('/Users/jamiecarnevale/Documents/Flatiron/Projects/Phase 1/Movie_Production_Recommendations/Jamie/data/CSV_DB_Merge_V2.csv')

# Further Cleaning Merged DataFrame
- This will help me explore the data more efficiently

In [ ]:
df = cleancm[['title_x', 'genres', 'original_language', 'production_budget', 'worldwide_gross', 
                     'averagerating', 'release_date_x', 'domestic_gross_x', 'foreign_gross_x',
                      'popularity', 'vote_average', 'runtime_minutes',
                     'vote_count', 'release_year', 'studio', 'numvotes',
                        ]]

In [ ]:
df = df.rename(columns={'title_x':'Title', 'genres': 'Genres', 'original_language':'OL', 
                   'production_budget': 'PBudget', 'worldwide_gross':'WWGross', 
                     'averagerating':'AvRating', 'release_date_x':'RDate', 
                   'domestic_gross_x':'DGrossX', 'foreign_gross_x':'FGrossX',
                      'popularity':'Pop', 'vote_average':'VoteAv', 'runtime_minutes':'RuntimeMin',
                     'vote_count':'VoteCount', 'release_year':'RelYear', 'studio':'Studio', 'numvotes':'NumVotes'})

In [ ]:
df['PBudget'] = df['PBudget'].str.replace('$','')
df['PBudget'] = df['PBudget'].str.replace(',','')
df['PBudget'] = df['PBudget'].astype(int)

In [ ]:
df['Profit'] =  df['WWGross'] - df['PBudget']

In [ ]:
df = df[['Title', 'Genres', 'OL', 
                   'PBudget', 'WWGross', 'Profit',
                     'AvRating', 'RDate', 
                   'DGrossX', 'FGrossX',
                      'Pop', 'VoteAv', 'RuntimeMin',
                     'VoteCount', 'RelYear', 'Studio', 'NumVotes']]

In [ ]:
df['IsProfitable'] = df['Profit'] > 0

In [ ]:
df.sort_values(by='Profit', ascending=False).tail(10)

In [ ]:
df.info()

In [ ]:
df

In [ ]:
dfexplode = df.copy()
dfexplode['Genres'] = df['Genres'].str.split(',')
dfexplode = dfexplode.explode('Genres')
dfexplode

In [ ]:
dfexplode.head()

# Exploring Clean Data Visually

In [ ]:
#list to change variables around: 
# 'Profit','Pop','Genres','WWGross','AvRating','DGrossX','FGrossX','RuntimeMin','Studio','PBudget','IsProfitable'
dfexplodeexplore = dfexplode[['Profit','Pop','Genres','AvRating','WWGross', 'RelYear',
                                'RuntimeMin','IsProfitable', 'PBudget']]
dfexplodeexplore

In [ ]:
sns.pairplot(dfexplodeexplore, hue='IsProfitable', plot_kws={'alpha':0.3}, palette={True:'lightblue',False:'darkred'})

## Just looking at negative profit

In [ ]:
negprofit = dfexplode[dfexplode['Profit'] < 0]
negprofit.head()

In [ ]:
negprofitexplore = negprofit[['Profit','Pop','WWGross','AvRating', 'Genres',
                                'DGrossX','FGrossX','RuntimeMin','Studio','PBudget']]

In [ ]:
sns.pairplot(negprofitexplore)

## Looking at Genres

In [ ]:
(dfexplode[df['Genres'].str.contains('Thriller')]).sort_values(by='Profit').tail(30)

In [ ]:
x = dfexplodeexplore['Genres'].value_counts()
y = x[x > 20].index.tolist()
dfexplodeexploregenres = dfexplodeexplore[dfexplodeexplore['Genres'].isin(y)]
dfexplodeexploregenres.head(1)

In [ ]:
negprofitexploregenres = dfexplodeexploregenres[dfexplodeexploregenres['Profit'] < 0]
negprofitexploregenres.shape

In [ ]:
print(dfexplodeexplore['Genres'].value_counts())

In [ ]:
print(dfexplodeexploregenres['Genres'].value_counts())
sns.catplot(data=dfexplodeexploregenres, x='Genres', y = 'Profit', kind='box', height=5, aspect=2)
plt.xticks(rotation=90)
plt.show()

In [ ]:
columns = ['Profit','Pop','WWGross','AvRating', 'RuntimeMin']
print(dfexplodeexploregenres['Genres'].value_counts())
for col in columns:
    sns.catplot(data=dfexplodeexploregenres, x='Genres', y = col, kind='box', height=5, aspect=2,)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
dfexplodeexplore.shape

In [ ]:
columns = ['Profit','Pop','WWGross','AvRating', 'RuntimeMin','Studio']
print(negprofitexploregenres['Genres'].value_counts())
for col in columns:
    sns.catplot(data=negprofitexploregenres, x='Genres', y = col, kind='strip', height=5, aspect=2)
    plt.xticks(rotation=90)
    plt.show()

## Lets make these graphs a bit more pretty

In [ ]:
pbg = dfexplodeexploregenres.groupby('Genres')['Profit'].median().sort_values(ascending=False)
dfexplodeexploregenres['Genres'] = pd.Categorical(dfexplodeexploregenres['Genres'], categories=pbg.index, ordered=True)
pbg

In [ ]:
dfexplodeexploregenres.groupby('Genres')['Profit'].mean().sort_values(ascending=False)


In [ ]:
pbg2 = negprofitexploregenres.groupby('Genres')['Profit'].median().sort_values()
negprofitexploregenres['Genres'] = pd.Categorical(negprofitexploregenres['Genres'], categories=pbg2.index, ordered=True)
pbg2

In [ ]:
sns.set_theme(style='whitegrid')

In [ ]:
print(dfexplodeexploregenres['Genres'].value_counts())
sns.catplot(data=dfexplodeexploregenres, x='Genres', y = 'Profit', kind='box', height=5, aspect=2, palette='viridis')
plt.xticks(rotation=90)
plt.title('Best Performing Genres by Profit')
plt.show()

In [ ]:
print(negprofitexploregenres['Genres'].value_counts())
sns.catplot(data=negprofitexploregenres, x='Genres', y = 'Profit', kind='swarm', height=5, aspect=2)
plt.title('Unprofitable Genres')
plt.xticks(rotation=90)
plt.show()

# Creating a Graph that tests the Proportionality between Profitable Films and Unprofitable Films by Genre

In [ ]:
#percent of IsProfitable False / IsProfitable.valuecount

In [ ]:
dfexplodeexploregenres.groupby('Genres')['IsProfitable'].count()

In [ ]:
x = dfexplodeexplore['Genres'].value_counts()

In [ ]:

x = dfexplodeexploregenres.groupby('Genres')['IsProfitable'].count().sort_values(ascending=False)
y = dfexplodeexploregenres.groupby('Genres')['IsProfitable'].sum().sort_values(ascending=False)
z = (x-y)/x
percentprofit = z.reset_index()
percentprofit.columns = ['Genre', 'PercentNotProfitable']
percentprofit['PercentNotProfitable'] = percentprofit['PercentNotProfitable'] * 100
percentprofit = percentprofit.sort_values(by='PercentNotProfitable', ascending=False)
percentprofit

In [ ]:
fig, ax =plt.subplots(figsize=(16, 10))
sns.barplot(data=percentprofit, x='Genre', y='PercentNotProfitable', order=percentprofit['Genre'], palette='viridis',ax=ax)
print(dfexplodeexploregenres.groupby('Genres')['IsProfitable'].count())
ax.set_title('Percent of Financially Draining Films by Genre')
plt.xticks(rotation=90)
plt.show()

In [ ]:
dfexplodeexploregenres